# Loop 9 Analysis: First Improvement Achieved!

After 8 experiments stuck at 70.676102, we finally achieved an improvement to 70.659944.

## Key Questions:
1. Which N values improved and by how much?
2. What's the potential for further improvement?
3. What approaches should we try next?

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from collections import defaultdict

# Load our best submission
submission = pd.read_csv('/home/submission/submission.csv', dtype=str)
print(f"Loaded submission with {len(submission)} rows")

Loaded submission with 20100 rows


In [2]:
# Load baseline for comparison
baseline = pd.read_csv('/home/code/experiments/001_baseline/santa-2025.csv', dtype=str)
print(f"Loaded baseline with {len(baseline)} rows")

Loaded baseline with 20100 rows


In [3]:
# Calculate scores for each N in both submission and baseline
from shapely.geometry import Polygon
from shapely import affinity

TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(x, y, deg):
    coords = list(zip(TX, TY))
    base_poly = Polygon(coords)
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def get_bounding_box_side(trees):
    if not trees:
        return float('inf')
    all_x, all_y = [], []
    for x, y, deg in trees:
        poly = get_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def load_trees_for_n(df, n):
    prefix = f'{n:03d}_'
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        trees.append((x, y, deg))
    return trees

print("Functions defined")

Functions defined


In [4]:
# Calculate per-N scores for both
baseline_scores = {}
submission_scores = {}

for n in range(1, 201):
    # Baseline
    trees = load_trees_for_n(baseline, n)
    if len(trees) == n:
        side = get_bounding_box_side(trees)
        baseline_scores[n] = side**2 / n
    
    # Submission
    trees = load_trees_for_n(submission, n)
    if len(trees) == n:
        side = get_bounding_box_side(trees)
        submission_scores[n] = side**2 / n

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"Submission total: {sum(submission_scores.values()):.6f}")
print(f"Improvement: {sum(baseline_scores.values()) - sum(submission_scores.values()):.6f}")

Baseline total: 70.676102
Submission total: 70.659944
Improvement: 0.016159


In [5]:
# Find which N values improved
improvements = []
for n in range(1, 201):
    if n in baseline_scores and n in submission_scores:
        diff = baseline_scores[n] - submission_scores[n]
        if diff > 0.0001:  # Significant improvement
            improvements.append({
                'n': n,
                'baseline': baseline_scores[n],
                'submission': submission_scores[n],
                'improvement': diff,
                'pct_improvement': 100 * diff / baseline_scores[n]
            })

improvements_df = pd.DataFrame(improvements)
improvements_df = improvements_df.sort_values('improvement', ascending=False)
print(f"\n{len(improvements_df)} N values improved:")
print(improvements_df.to_string(index=False))


24 N values improved:
  n  baseline  submission  improvement  pct_improvement
 57  0.358045    0.354108     0.003937         1.099478
 54  0.361321    0.359200     0.002120         0.586846
101  0.350389    0.349036     0.001353         0.386284
162  0.338332    0.337058     0.001274         0.376609
 74  0.354139    0.353127     0.001012         0.285744
 75  0.353773    0.352898     0.000875         0.247342
123  0.348717    0.347904     0.000813         0.233070
157  0.341876    0.341371     0.000505         0.147678
187  0.340604    0.340237     0.000367         0.107780
195  0.332901    0.332610     0.000291         0.087439
141  0.343724    0.343470     0.000254         0.073986
 76  0.351603    0.351372     0.000231         0.065592
143  0.341362    0.341148     0.000214         0.062735
142  0.341339    0.341148     0.000192         0.056128
193  0.333950    0.333763     0.000187         0.055981
 92  0.348482    0.348310     0.000172         0.049279
200  0.337731    0.337564

In [6]:
# Analyze potential for further improvement
print("\n=== Gap Analysis ===")
print(f"Current score: 70.659944")
print(f"Target score: 68.919154")
print(f"Gap: {70.659944 - 68.919154:.6f} points ({100*(70.659944 - 68.919154)/70.659944:.2f}%)")

# If we improved 24 N values by 0.016, how many more improvements do we need?
print(f"\nImprovement achieved: 0.016158 points")
print(f"Improvements needed: {(70.659944 - 68.919154) / 0.016158:.1f}x more")
print(f"\nThis means we need to find ~108x more improvement!")
print("\nPossible strategies:")
print("1. Backward iteration (BackPacking) - propagate good configs from larger N to smaller N")
print("2. Targeted SA on improved N values - they may be in better basins")
print("3. Find more CSV sources with better configurations")
print("4. Symmetric packing patterns for specific N values")


=== Gap Analysis ===
Current score: 70.659944
Target score: 68.919154
Gap: 1.740790 points (2.46%)

Improvement achieved: 0.016158 points
Improvements needed: 107.7x more

This means we need to find ~108x more improvement!

Possible strategies:
1. Backward iteration (BackPacking) - propagate good configs from larger N to smaller N
2. Targeted SA on improved N values - they may be in better basins
3. Find more CSV sources with better configurations
4. Symmetric packing patterns for specific N values


In [7]:
# Check which N values have the most room for improvement
# Compare to theoretical minimum (packing efficiency)
print("\n=== Per-N Score Analysis ===")
print("\nN values with highest scores (most room for improvement):")
scores_list = [(n, submission_scores[n]) for n in range(1, 201)]
scores_list.sort(key=lambda x: x[1], reverse=True)
for n, score in scores_list[:20]:
    print(f"  N={n}: score={score:.6f}")


=== Per-N Score Analysis ===

N values with highest scores (most room for improvement):
  N=1: score=0.661250
  N=2: score=0.450779
  N=3: score=0.434745
  N=5: score=0.416850
  N=4: score=0.416545
  N=7: score=0.399897
  N=6: score=0.399610
  N=9: score=0.387415
  N=8: score=0.385407
  N=15: score=0.379203
  N=10: score=0.376630
  N=21: score=0.376451
  N=20: score=0.376057
  N=11: score=0.375736
  N=22: score=0.375258
  N=16: score=0.374128
  N=26: score=0.373997
  N=12: score=0.372724
  N=13: score=0.372294
  N=25: score=0.372144


In [8]:
# Check N values that DIDN'T improve - are they already optimal?
print("\n=== N values that didn't improve ===")
no_improvement = []
for n in range(1, 201):
    if n in baseline_scores and n in submission_scores:
        diff = baseline_scores[n] - submission_scores[n]
        if diff <= 0.0001:
            no_improvement.append(n)

print(f"{len(no_improvement)} N values showed no improvement")
print(f"\nSmall N (1-20) that didn't improve: {[n for n in no_improvement if n <= 20]}")
print(f"These are likely already at or near optimal.")


=== N values that didn't improve ===
176 N values showed no improvement

Small N (1-20) that didn't improve: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20]
These are likely already at or near optimal.
